In [ ]:
from getpass import getpass
import pytds
from pytds.login import NtlmAuth
from pathlib import Path
from collections import defaultdict
import time
password = ''

In [ ]:
host=''
myuser = ''
dump_queries =True
sleep = 0.25

In [ ]:
ignore_users = [myuser, 'NT AUTHORITY\\SYSTEM']

In [ ]:
if password == '':
    password = getpass('please enter your password')

In [ ]:
def run_query(query, host, database=None):
    global password
    with pytds.connect(
        server=host,
        database=database,
        auth=NtlmAuth(user_name="ingham-county\\srudloff",password=password),
        as_dict=True,
    ) as conn:
        with conn.cursor() as cur: 
            cur.execute(query)

            c = True
            while c is not None:
                try:
                    return list(cur.fetchall())
                except Exception as e:
                    c = cur.nextset()

In [ ]:
for file in Path('sql_dumps').iterdir():
    if file.is_file():
        file.unlink()

In [ ]:
def getRunningQueries(host):
    return run_query("""SELECT
        p.spid, p.status, p.hostname, p.loginame, p.cpu, r.start_time, r.command,
        p.program_name, text 
    FROM
        sys.dm_exec_requests AS r,
        master.dbo.sysprocesses AS p 
        CROSS APPLY sys.dm_exec_sql_text(p.sql_handle)
    WHERE
        p.status NOT IN ('sleeping', 'background') 
    AND r.session_id = p.spid""",host=host)

In [ ]:
i = defaultdict(int)
runs = 0
known = []
while True:
    runs+=1
    if runs % 100 == 0:
        print(f'{runs} runs')
    try:
        for query in getRunningQueries(host=host):
            if query not in known:
                if query['loginame'].strip() in ignore_users:
                    continue
                for item in ['hostname', 'status', 'loginame', 'command', 'program_name', 'start_time']:
                    print(str(query[item]).strip(), end='\t')
                if dump_queries:
                    fn = f'query_{query["spid"]}_{i[query["spid"]]}.sql'
                    print(f"dump: {fn}")
                    with open(f"sql_dumps/{fn}", 'w') as f:
                        f.write(query['text'])
                    i[query['spid']] += 1
                known.append(query)
    except Exception as e:
        print(e)
        time.sleep(1)
    time.sleep(sleep)